In [1]:
import os
import sys
import pandas as pd

df1 = pd.read_excel(r"C:\Users\evany_cdhq038\OneDrive\Desktop\1_21_2023_Lexus\Moody's LinkedIn Analysts Sets 1+2.xlsx", sheet_name='Moodys LinkedIn Analysts 1')
#df2 = pd.read_excel(r"C:\Users\evany_cdhq038\OneDrive\Desktop\1_21_2023_Lexus\Moody's LinkedIn Analysts Sets 1+2.xlsx", sheet_name='Moodys LinkedIn Analysts 2')


In [2]:
#cleaning dfs to remove LexusMatch of 0
#df1 = df1[df1['lexismatch'] == 1.0]
#df2 = df2[df2['lexismatch'] == 1.0]
oneDF=df1[df1['lexismatch'] == 1.0]
zeroDF=df1[df1['lexismatch'] != 1.0]
print(oneDF)

     mdanid  lexismatch                Analyst  \
0         1           1         A.J. Sabatelle   
3         4           1        Denise Rappmund   
4         5           1      Earl Heffintrayer   
5         6           1            Julie Meyer   
6         7           1             Roy Ousley   
..      ...         ...                    ...   
828     829           1  Yasmine Grossenbacher   
831     832           1       Yoshio Takahashi   
832     833           1           Yves J Lemay   
833     834           1             Yves Lemay   
834     835           1           Zev Halstuch   

                                          linkedin_url  \
0    https://www.linkedin.com/in/angelo-sabatelle-9...   
3    https://www.linkedin.com/in/denise-rappmund-47...   
4    https://www.linkedin.com/in/earl-heffintrayer-...   
5     https://www.linkedin.com/in/julieelizabethmeyer/   
6               https://www.linkedin.com/in/royousley/   
..                                                 

In [3]:
print(zeroDF)

     mdanid  lexismatch                   Analyst  \
1         2           0              Adam McLaren   
2         3           0           Amol Joshi, CFA   
8         9           0             Andrew Brooks   
15       16           0  Bartosz Krzysztof. Walas   
16       17           0           Benjamin Nelson   
..      ...         ...                       ...   
826     827           0               Yaron Ernst   
827     828           0          Yaroslav Sovgyra   
829     830           0              Yasunobu Doi   
830     831           0             Yian Ning Loh   
835     836           0               Zhenyu Zhao   

                                          linkedin_url  \
1                                                  NaN   
2                https://www.linkedin.com/in/ajoshius/   
8    https://www.linkedin.com/in/andrew-brooks-b845...   
15          https://www.linkedin.com/in/bartosz-walas/   
16     https://www.linkedin.com/in/ben-nelson-21a8b64/   
..             

In [4]:
#fuzzy matching to pdfname
from fuzzywuzzy import fuzz
#value = fuzz.ratio('New York', 'New York')
analystNames= oneDF["Analyst"]
cleanedAnalystNames=[]
purePDFNames=[]
#cleanedPDFNames=[]
correspondingPDFs=[]
correspondingMatchRatio=[]
pdfMatch=[]

import os
root = "C:\\Users\\evany_cdhq038\\OneDrive\\Desktop\\1_21_2023_Lexus\\Elaine Han Moody's Lexis Nexis PDFs Sets 1+2"
for entry in os.scandir(root):
    purePDFNames.append(entry.name)
    #cleanedPDFNames.append(entry.name[:-4].replace("_", " "))
    
#removing some extra files
purePDFNames.pop(0) #removes ipynb checkpoint
purePDFNames.pop(0) #removes program in the folder

#cleaning
for dirtyName in analystNames:
    dirtyName=dirtyName.replace(",","")
    dirtyName=dirtyName.replace("CFA","")
    dirtyName=dirtyName.replace("(","")
    dirtyName=dirtyName.replace(")","")
    cleanedAnalystNames.append(dirtyName)
    
#splitting into first and last name
#finding the one with the highest geometric average
for name in cleanedAnalystNames:
    name=name.strip()
    firstName=name[:name.find(" ")].strip().lower()
    lastName=name[name.rfind(" "):].strip().lower()
    print(firstName+" : "+ lastName)
    firstNameRatio=1
    lastNameRatio=1
    highestMatchValue=0
    highestMatchIndex=0
    
    i=0
    while i<len(purePDFNames):
        PDF=purePDFNames[i]
        first=PDF[:PDF.find("_")].strip().lower()
        last=PDF[PDF.find("_")+1:PDF.find(".")].strip().lower()
        
        firstNameRatio=fuzz.ratio(first,firstName)
        lastNameRatio=fuzz.ratio(last,lastName)
        if firstNameRatio==0:
            firstNameRatio+=1
            
        if lastNameRatio==0:
            lastNameRatio+=1
            
        geomAvg=(int(firstNameRatio)*int(lastNameRatio))/(int(firstNameRatio)+int(lastNameRatio))
        if geomAvg>highestMatchValue:
            highestMatchValue=geomAvg
            highestMatchIndex=i
        i+=1
        
    
    correspondingPDFs.append(purePDFNames[highestMatchIndex])
    correspondingMatchRatio.append(highestMatchValue)
    


#oneDF["PDFMatch>=80"]=pdfMatch
oneDF["correspondingPDF"]=correspondingPDFs
oneDF["correspondingMatchRatio"]=correspondingMatchRatio

oneDF=oneDF.sort_values(by='correspondingMatchRatio', ascending=False)
columnHeaders=['mdanid', 'lexismatch','Analyst','correspondingPDF','correspondingMatchRatio','linkedin_url','linkedin_title','Group','office','gender','mis_years?','first_job_start_year','grad_instit','grad_degree','grad_year','ug_instit','ug_year','comments','Division','Country','namelen','linkedin_name','title','moody_start','moody_end','source']
oneDF = oneDF[columnHeaders]

C:\Users\evany_cdhq038\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


a.j. : sabatelle
denise : rappmund
earl : heffintrayer
julie : meyer
roy : ousley
anastasija : johnson
andrew : macdonald
aniello : infante
anna : zubets-anderson
arvinder : saluja
austin : custodio
austin : langer
botir : sharipov
brian : silver
bruce : clark
bruce : herskovics
calyn : gelinas
carl : salas
charles : o'shea
chedly : louis
christian : azzi
christina : padgett
cintia : nazima
daniel : gates
daniel : greszta
david : keisman
derek : gluckman
dilara : sukhov
dylan : duzey
edmond : deforest
emile : nems
enam : hoque
eoin : roche
eric : greaser
eric : kozlowski
fernando : freijedo
gail : sussman
gayle : podurgiel
gerald : granovsky
giovanni : velez
glenn : eckert
gretchen : french
gurkirat : hans
ignacio : rasero
jack : leyland
jadijhe : adamo
jairo : chung
janice : hofferber
jean-yves : coupin
jeffrey : cassella
jessica : gladstone
jessica : raab
jim : hempstead
joanna : o'brien
joe : tringali
john : puchalla
john : rogers
john : thieroff
jonathan : root
juan : acosta
justin

<ipython-input-4-1cea767edee3>:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oneDF["correspondingPDF"]=correspondingPDFs
<ipython-input-4-1cea767edee3>:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oneDF["correspondingMatchRatio"]=correspondingMatchRatio


In [5]:
import openpyxl


oneDF.to_excel("2_15_EvanMoody1.xlsx")



In [6]:
zeroDF.to_excel("2_15_EvanMoody0.xlsx")

In [ ]:
#other stuff below

In [35]:
oneDF.columns.values.tolist()

['mdanid',
 'lexismatch',
 'Analyst',
 'linkedin_url',
 'Title',
 'Group',
 'office',
 'gender',
 'mis_years?',
 'first_job_start_year',
 'grad_instit',
 'grad_degree',
 'grad_year',
 'ug_instit',
 'ug_year',
 'Age',
 'linkedin_url.1',
 'comments',
 'Division',
 'Country',
 'namelen',
 'linkedin_name',
 'title',
 'moody_start',
 'moody_end',
 'correspondingPDF',
 'correspondingMatchRatio']

In [3]:
test="Evan Yang, "
test= test.replace("CFA","")
print(test)

Evan Yang, 


In [5]:
#fuzzy matching to pdfname

analystNames= df2["Analyst"]
purePDFNames=[]
cleanedPDFNames=[]
correspondingPDFs=[]
correspondingMatchRatio=[]

import os
root = "C:\\Users\\evany_cdhq038\\OneDrive\\Desktop\\1_21_2023_Lexus\\Elaine Han Moody's Lexis Nexis PDFs Sets 1+2"
for entry in os.scandir(root):
    purePDFNames.append(entry.name)
    cleanedPDFNames.append(entry.name[:-4].replace("_", " "))
    

for name in analystNames:
    highestMatchValue=0
    highestMatchIndex=0
    i=0
    while i<len(purePDFNames):
        if fuzz.ratio(name,cleanedPDFNames[i])>highestMatchValue:
            highestMatchIndex=i
            highestMatchValue=fuzz.ratio(name,cleanedPDFNames[i])
        i+=1
    correspondingPDFs.append(purePDFNames[highestMatchIndex])
    correspondingMatchRatio.append(highestMatchValue)

df2["correspondingPDF"]=correspondingPDFs
df2["correspondingMatchRatio"]=correspondingMatchRatio

In [6]:

df2.to_excel("Evan_Moodys_Set_2.xlsx")


In [14]:
for PDF in purePDFNames:
    first=PDF[:PDF.find("_")]
    last=PDF[PDF.find("_")+1:PDF.find(".")]
    print(first)
    print(last)

.ipynb

1
28_2023_Extractor
Aaron
Freedman
Alan
Murray
Alejandro
Olivo
Alexander
Yavorsky
Alexandra
Parker
Alina
Khavulya
Allen
Tischler
Anastasija
Johnson
Andrew
Forsyth
Andrew
MacDonald
Andrew
Oram
Angela
Burriesci
Angela
Jameson
Angelo
Sabatelle
Aniello
Infante
Anna
Sherbakova
Anna
Zubets-Anderson
Anthony
Hill
Arif
Bekiroglu
Arlene
Isaacs-Lowe
Arthur
Fliegelman
Arvinder
Saluja
Ashley
Staropoli
Austin
Custodio
Austin
Langer
Barry
Wadler
Bart
Oosterveld
Bob
Garofalo
Botir
Sharapov
Brian
Oak
Brian
Silver
Brian
Weddington
Bruce
Clark
Bruce
Herskovics
Calyn
Gelinas
Carl
Salas
Catherine
Guinee
Cecilia
Lam
Charles
O&#39Shea
Charles
O_Shea
Charles
Tan
Chedley
Louis
Cheryl
Solometo
Chetan
Modi
Christian
Azzi
Christina
padgett
Christopher
O&#39Neil
Christophe
Razaire
Chris
Park
Chris
Wimmer (1)
Chris
Wimmer
Cintia
Nazima
Clifford
Crain
Constanze
Rheinbaben
Costas
Chrysostomou
Craig
Emrick
Daniel
Gates
Daniel
Greszta
Daniel
Marx
Daniel
Serrao
David
Berge
David
Fanger (1)
David
Fanger
David
Ham

In [15]:
print(cleanedPDFNames)

['.ipynb checkpo', '1 28 2023 Extractor.i', 'Aaron Freedman', 'Alan Murray', 'Alejandro Olivo', 'Alexander Yavorsky', 'Alexandra Parker', 'Alina Khavulya', 'Allen Tischler', 'Anastasija Johnson', 'Andrew Forsyth', 'Andrew MacDonald', 'Andrew Oram', 'Angela Burriesci', 'Angela Jameson', 'Angelo Sabatelle', 'Aniello Infante', 'Anna Sherbakova', 'Anna Zubets-Anderson', 'Anthony Hill', 'Arif Bekiroglu', 'Arlene Isaacs-Lowe', 'Arthur Fliegelman', 'Arvinder Saluja', 'Ashley Staropoli', 'Austin Custodio', 'Austin Langer', 'Barry Wadler', 'Bart Oosterveld', 'Bob Garofalo', 'Botir Sharapov', 'Brian Oak', 'Brian Silver', 'Brian Weddington', 'Bruce Clark', 'Bruce Herskovics', 'Calyn Gelinas', 'Carl Salas', 'Catherine Guinee', 'Cecilia Lam', 'Charles O&#39Shea', 'Charles O Shea', 'Charles Tan', 'Chedley Louis', 'Cheryl Solometo', 'Chetan Modi', 'Christian Azzi', 'Christina padgett', 'Christopher O&#39Neil', 'Christophe Razaire', 'Chris Park', 'Chris Wimmer (1)', 'Chris Wimmer', 'Cintia Nazima', 'C

In [ ]:
for name in cleanedAnalystNames:
    print(name)
    highestMatchValue=0
    highestMatchIndex=0
    i=0
    while i<len(purePDFNames):
        if fuzz.ratio(name,cleanedPDFNames[i])>highestMatchValue:
            highestMatchIndex=i
            highestMatchValue=fuzz.ratio(name,cleanedPDFNames[i])
        i+=1
    correspondingPDFs.append(purePDFNames[highestMatchIndex])
    correspondingMatchRatio.append(highestMatchValue)
    if highestMatchValue>=80:
        pdfMatch.append("1")
    else:
        pdfMatch.append("0")

In [34]:
purePDFNames.pop(0)
purePDFNames.pop(0)
print(purePDFNames)

['Alan_Murray.pdf', 'Alejandro_Olivo.pdf', 'Alexander_Yavorsky.pdf', 'Alexandra_Parker.pdf', 'Alina_Khavulya.pdf', 'Allen_Tischler.pdf', 'Anastasija_Johnson.pdf', 'Andrew_Forsyth.pdf', 'Andrew_MacDonald.pdf', 'Andrew_Oram.pdf', 'Angela_Burriesci.pdf', 'Angela_Jameson.pdf', 'Angelo_Sabatelle.pdf', 'Aniello_Infante.pdf', 'Anna_Sherbakova.pdf', 'Anna_Zubets-Anderson.pdf', 'Anthony_Hill.pdf', 'Arif_Bekiroglu.pdf', 'Arlene_Isaacs-Lowe.pdf', 'Arthur_Fliegelman.pdf', 'Arvinder_Saluja.pdf', 'Ashley_Staropoli.pdf', 'Austin_Custodio.pdf', 'Austin_Langer.pdf', 'Barry_Wadler.pdf', 'Bart_Oosterveld.pdf', 'Bob_Garofalo.pdf', 'Botir_Sharapov.pdf', 'Brian_Oak.pdf', 'Brian_Silver.pdf', 'Brian_Weddington.pdf', 'Bruce_Clark.pdf', 'Bruce_Herskovics.pdf', 'Calyn_Gelinas.pdf', 'Carl_Salas.pdf', 'Catherine_Guinee.pdf', 'Cecilia_Lam.pdf', 'Charles_O&#39Shea.pdf', 'Charles_O_Shea.pdf', 'Charles_Tan.pdf', 'Chedley_Louis.pdf', 'Cheryl_Solometo.pdf', 'Chetan_Modi.pdf', 'Christian_Azzi.pdf', 'Christina_padgett.pdf